In [2]:
# import matplotib

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import csv

In [ ]:
# import data

ROOT_PATH = "RC Lab data Raw/"

SUB_FOLDERS = ["change cap", "change resistor", "change voltage"]

def read_data(file_path):
    data = pd.read_csv(file_path, sep=',', header=0)
    df = pd.DataFrame(data)
    return df

def plot_data(df, title, x_label, y_label):
    plt.figure()
    plt.plot(df[' time'], df[' cal[0]'], label='Voltage', color='blue')
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.show()

def Trim_data(df):
    non_zero_indices = df[df[' cal[0]'] != 0].index
    start_index = non_zero_indices.min()
    end_index = non_zero_indices.max()
    df_trimmed = df.loc[start_index:end_index]
    return df_trimmed

def combine_data_to_single_time_series(df1, df2):
    # Ensure only the required columns are present before renaming
    df1 = df1[[' time', ' cal[0]']]
    df2 = df2[[' time', ' cal[0]']]
    # rename the columns of the dataframes to avoid confusion
    df1.columns = [' time', ' cal[0]_1']
    df2.columns = [' time', ' cal[0]_2']
    df_combined = pd.merge_asof(df1.sort_values(' time'), df2.sort_values(' time'), on=' time', suffixes=('_1', '_2'))
    return df_combined

def main():
    for folder in SUB_FOLDERS:
        folder_path = ROOT_PATH + folder
        files = os.listdir(folder_path)
        for i in range(0, len(files), 2):
            file_path_1 = folder_path + "/" + files[i]
            file_path_2 = folder_path + "/" + files[i + 1]
            
            data_1 = read_data(file_path_1)
            data_1 = Trim_data(data_1)
            
            data_2 = read_data(file_path_2)
            data_2 = Trim_data(data_2)
            
            combined_data = combine_data_to_single_time_series(data_1, data_2)
            
            plot_data(combined_data, "Combined Voltage vs Time", "Time", "Voltage")


if __name__ == "__main__":
    main()
